In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, accuracy_score

In [3]:
all_proteins = pd.read_csv("protein_augmentation_datasets.csv")


# Excludes proteins which contain selenocysteine

all_proteins = all_proteins[all_proteins['Sequence'].str.contains('U') == False]


# Creates the separate task datasets

tclin_set = all_proteins[['Sequence', 'Tclin']].copy().rename(
    columns={"Tclin": "Label"})
metal_ion_set = all_proteins[['Sequence', 'Metal ion binding']].copy().rename(
    columns={"Metal ion binding": "Label"})
pol_II_set = all_proteins[['Sequence', 'Pos reg RNA pol II']].copy().rename(
    columns={"Pos reg RNA pol II": "Label"})
colitis_set = all_proteins[['Sequence', 'Ulcerative colitis']].copy().rename(
    columns={"Ulcerative colitis": "Label"})
lc_set = all_proteins[['Sequence', 'Liver cancer']].copy().rename(
    columns={"Liver cancer": "Label"})

all_proteins

,UniProt,Symbol,Sequence,Tclin,Metal ion binding,Pos reg RNA pol II,Ulcerative colitis,Liver cancer
0,P32929,CTH,MQEKDASSQGFLPHFQHFATQAIHVGQDPEQWTSRAVVPPISLSTT...,0,0,0,0,0
1,A4D0Y5,C7orf77,MGAERVCTKAPEITQDEAEIYSLTNMEGNIGIKGCEFKSWLFKFYQ...,0,0,0,0,0
2,Q49A92,C8orf34,MSSPLASELSELAALRPGFRLSAPHARVAPRAATHARGRGRASHAG...,0,0,0,0,0
3,Q9UFW8,CGGBP1,MERFVVTAPPARNRSKTALYVTPLDRVTEFGGELHEDGGKLFCTSC...,0,0,0,0,0
4,Q96K31,C8orf76,MDSGCWLFGGEFEDSVFEERPERRSGPPASYCAKLCEPQWFYEETE...,0,0,0,0,0
...,...,...,...,...,...,...,...,...
20406,P49286,MTNR1B,MSENGSFANCCEAGGWAVRPGWSGAGSARPSRTPRPPWVAPALSAV...,1,0,0,0,0
20407,P84157,MXRA7,MEAPAELLAALPALATALALLLAWLLVRRGAAASPEPARAPPEPAP...,0,0,0,1,0
20408,Q9NP71,MLXIPL,MAGALAGLAAGLQVPRVAPSPDSDSDTDSEDPSLRRSAGGLLRSQV...,0,0,1,0,0
20409,Q9BWT6,MND1,MSKKKGLSAEEKRTRMMEIFSETKDVFQLKDLEKIAPKEKGITAMS...,0,0,0,0,0


In [4]:
# Initializes and fits the tokenizer for amino acid sequences

aa_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
aa_tokenizer.fit_on_texts(all_proteins['Sequence'])


# Initializes the nucleotide sequence tokenizer and provides it with the 61 
# codons

nt_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=False, 
                                                     lower=False)
nt_tokenizer.word_index = {'GCT':1, 'GCC':2, 'GCA':3, 'GCG':4, 'CGT':5, 'CGC':6,
                           'CGA':7, 'CGG':8, 'AGA':9, 'AGG':10, 'AAT':11, 
                           'AAC':12, 'GAT':13, 'GAC':14,'TGT':15, 'TGC':16, 
                           'CAA':17, 'CAG':18, 'GAA':19, 'GAG':20, 'GGT':21, 
                           'GGC':22, 'GGA':23, 'GGG':24, 'CAT':25, 'CAC':26, 
                           'ATT':27, 'ATC':28, 'ATA':29, 'CTT':30, 'CTC':31, 
                           'CTA':32, 'CTG':33, 'TTA':34, 'TTG':35, 'AAA':36, 
                           'AAG':37, 'ATG':38, 'TTT':39, 'TTC':40, 'CCT':41, 
                           'CCC':42, 'CCA':43, 'CCG':44, 'TCT':45, 'TCC':46, 
                           'TCA':47, 'TCG':48, 'AGT':49, 'AGC':50, 'ACT':51, 
                           'ACC':52, 'ACA':53, 'ACG':54, 'TGG':55, 'TAT':56, 
                           'TAC':57, 'GTT':58, 'GTC':59, 'GTA':60, 'GTG':61}

In [13]:
def vanilla_aa_oversample(train_set, test_set, max_len=512):
  """
  Truncates the sequences in the training and test sets, augments the 
  training set by simple oversampling with replacement, tokenizes and pads the
  sequences, and returns the datasets.

  Args:
    train_set (DataFrame): Pandas dataframe containing the training set
    test_set (DataFrame): Pandas dataframe containing the test set
    max_len (int): maximum sequence length to truncate to
  
  Returns:
    train_copy (DataFrame): a Pandas dataframe containing the oversampled 
    training data
    test_copy (DataFrame): a Pandas dataframe containing the test data

  """

  train_copy = train_set.copy()
  test_copy = test_set.copy()

  train_copy['Sequence'] = train_copy['Sequence'].apply(lambda x: x[-max_len:])
  test_copy['Sequence'] = test_copy['Sequence'].apply(lambda x: x[-max_len:])

  train_pos = train_copy[train_copy['Label'] == 1]
  train_neg = train_copy[train_copy['Label'] == 0]

  train_pos = train_pos.sample(n=len(train_neg), replace=True)

  train_copy = pd.concat([train_pos, train_neg])

  train_sequences = aa_tokenizer.texts_to_sequences(train_copy['Sequence'])
  test_sequences = aa_tokenizer.texts_to_sequences(test_copy['Sequence'])

  train_sequences = tf.keras.utils.pad_sequences(train_sequences,
                                                 maxlen=max_len)
  test_sequences = tf.keras.utils.pad_sequences(test_sequences,
                                                maxlen=max_len)

  train_copy['Sequence'] = [seq for seq in train_sequences]
  test_copy['Sequence'] = [seq for seq in test_sequences]

  return train_copy, test_copy


def ala_aa_oversample(train_set, test_set, max_len=512, p_sub=0.05):
  """
  Truncates the sequences in the training and test sets, augments the 
  training set by oversampling with replacement with alanine substitution of 
  random amino acids, tokenizes and pads the sequences, and returns the 
  datasets.

  Args:
    train_set (DataFrame): Pandas dataframe containing the training set
    test_set (DataFrame): Pandas dataframe containing the test set
    max_len (int): maximum sequence length to truncate to
    p_sub (float): probability of substituting each amino acid with alanine
  
  Returns:
    train_copy (DataFrame): a Pandas dataframe containing the oversampled 
    training data
    test_copy (DataFrame): a Pandas dataframe containing the test data
    
  """

  train_copy = train_set.copy()
  test_copy = test_set.copy()

  train_copy['Sequence'] = train_copy['Sequence'].apply(lambda x: x[-max_len:])
  test_copy['Sequence'] = test_copy['Sequence'].apply(lambda x: x[-max_len:])

  train_pos = train_copy[train_copy['Label'] == 1]
  train_neg = train_copy[train_copy['Label'] == 0]

  train_pos = train_pos.sample(n=len(train_neg), replace=True)

  pos_lists = list(train_pos['Sequence'])

  pos_new = []


  # Loops through each amino acid in each protein sequence and adds it to a new
  # sequence, or replaces with alanine with probability p_sub

  for i in range(len(pos_lists)):
    changed_seq = ""

    for j in range(len(pos_lists[i])):
      if np.random.uniform() < p_sub:
        changed_seq += 'A'
      
      else:
        changed_seq += pos_lists[i][j]

    pos_lists[i] = changed_seq

  train_pos['Sequence'] = [seq for seq in pos_lists]

  train_copy = pd.concat([train_pos, train_neg])

  train_sequences = aa_tokenizer.texts_to_sequences(train_copy['Sequence'])
  test_sequences = aa_tokenizer.texts_to_sequences(test_copy['Sequence'])

  train_sequences = tf.keras.utils.pad_sequences(train_sequences,
                                                 maxlen=max_len)
  test_sequences = tf.keras.utils.pad_sequences(test_sequences,
                                                maxlen=max_len)

  train_copy['Sequence'] = [seq for seq in train_sequences]
  test_copy['Sequence'] = [seq for seq in test_sequences]

  return train_copy, test_copy


def dict_aa_oversample(train_set, test_set, max_len=512, p_sub=0.05):
  """
  Truncates the sequences in the training and test sets, augments the 
  training set by oversampling with replacement with dictionary substitution of 
  random amino acids, tokenizes and pads the sequences, and returns the 
  datasets.

  Args:
    train_set (DataFrame): Pandas dataframe containing the training set
    test_set (DataFrame): Pandas dataframe containing the test set
    max_len (int): maximum sequence length to truncate to
    p_sub (float): probability of substituting each amino acid with a close 
    equivalent
  
  Returns:
    train_copy (DataFrame): a Pandas dataframe containing the oversampled 
    training data
    test_copy (DataFrame): a Pandas dataframe containing the test data
    
  """

  train_copy = train_set.copy()
  test_copy = test_set.copy()

  train_copy['Sequence'] = train_copy['Sequence'].apply(lambda x: x[-max_len:])
  test_copy['Sequence'] = test_copy['Sequence'].apply(lambda x: x[-max_len:])

  train_pos = train_copy[train_copy['Label'] == 1]
  train_neg = train_copy[train_copy['Label'] == 0]

  train_pos = train_pos.sample(n=len(train_neg), replace=True)

  pos_lists = list(train_pos['Sequence'])

  pos_new = []

  aa_sub_dict = {'A':'V', 'S':'T', 'F':'Y', 'K':'R', 'C':'M', 'D':'E', 'N':'Q', 
            'V':'I', 'V':'A', 'T':'S', 'Y':'F', 'R':'K', 'M':'C', 'E':'D', 
            'Q':'N', 'I':'V'}

  # Loops through each amino acid in each protein sequence and adds it to a new
  # sequence, or replaces with a similar amino acid via aa_sub_dict with 
  # probability p_sub

  for i in range(len(pos_lists)):
    changed_seq = ""

    for j in range(len(pos_lists[i])):
      if np.random.uniform() < p_sub and pos_lists[i][j] in aa_sub_dict:
        changed_seq += aa_sub_dict[pos_lists[i][j]]
      
      else:
        changed_seq += pos_lists[i][j]

    pos_lists[i] = changed_seq

  train_pos['Sequence'] = [seq for seq in pos_lists]

  train_copy = pd.concat([train_pos, train_neg])

  train_sequences = aa_tokenizer.texts_to_sequences(train_copy['Sequence'])
  test_sequences = aa_tokenizer.texts_to_sequences(test_copy['Sequence'])

  train_sequences = tf.keras.utils.pad_sequences(train_sequences,
                                                 maxlen=max_len)
  test_sequences = tf.keras.utils.pad_sequences(test_sequences,
                                                maxlen=max_len)

  train_copy['Sequence'] = [seq for seq in train_sequences]
  test_copy['Sequence'] = [seq for seq in test_sequences]

  return train_copy, test_copy


def aa_to_nt(seqs):
  """
  Takes a set of amino acid sequences and replaces each randomly with one of its
  corresponding nucleotide codons.

  Args:
    seqs (DataFrame): Pandas dataframe containing the sequences to be 
    substituted with nucleotide codons
  
  Returns:
    seq_lists (list): a list containing the new sequences as a list of strings,
    each string representing a codon
    
  """

  seq_lists = list(seqs)

  seqs_new = []

  nt_sub_dict = {
        'A': ['GCT ', 'GCC ', 'GCA ', 'GCG '],
        'R': ['CGT ', 'CGC ', 'CGA ', 'CGG ', 'AGA ', 'AGG '],
        'N': ['AAT ', 'AAC '],
        'D': ['GAT ', 'GAC '],
        'C': ['TGT ', 'TGC '],
        'Q': ['CAA ', 'CAG '],
        'E': ['GAA ', 'GAG '],
        'G': ['GGT ', 'GGC ', 'GGA ', 'GGG '],
        'H': ['CAT ', 'CAC '],
        'I': ['ATT ', 'ATC ', 'ATA '],
        'L': ['CTT ', 'CTC ', 'CTA ', 'CTG ', 'TTA ', 'TTG '],
        'K': ['AAA ', 'AAG '],
        'M': ['ATG '],
        'F': ['TTT ', 'TTC '],
        'P': ['CCT ', 'CCC ', 'CCA ', 'CCG '],
        'S': ['TCT ', 'TCC ', 'TCA ', 'TCG ', 'AGT ', 'AGC '],
        'T': ['ACT ', 'ACC ', 'ACA ', 'ACG '],
        'W': ['TGG '],
        'Y': ['TAT ', 'TAC '],
        'V': ['GTT ', 'GTC ', 'GTA ', 'GTG ']}

  # Loops through each amino acid in each protein sequence and replaces it with
  # a randomly chosen codon which codes for it from nt_sub_dict

  for i in range(len(seq_lists)):
    changed_seq = ""

    for j in range(len(seq_lists[i])):
      codon_index = np.random.randint(len(nt_sub_dict[seq_lists[i][j]]))

      codon = nt_sub_dict[seq_lists[i][j]][codon_index]

      changed_seq += codon
      
    seq_lists[i] = changed_seq
  
  return seq_lists


def nt_oversample(train_set, test_set, max_len=512):
  """
  Truncates the sequences in the training and test sets, converts amino acids
  randomly to corresponding nucleotide codons while oversampling the training
  set, tokenizes and pads the sequences, and returns the datasets.

  Args:
    train_set (DataFrame): Pandas dataframe containing the training set
    test_set (DataFrame): Pandas dataframe containing the test set
    max_len (int): maximum sequence length to truncate to
    p_sub (float): probability of substituting each amino acid with a close 
    equivalent
  
  Returns:
    train_copy (DataFrame): a Pandas dataframe containing the oversampled 
    training data
    test_copy (DataFrame): a Pandas dataframe containing the test data
    
  """

  train_copy = train_set.copy()
  test_copy = test_set.copy()

  train_copy['Sequence'] = train_copy['Sequence'].apply(lambda x: x[-max_len:])
  test_copy['Sequence'] = test_copy['Sequence'].apply(lambda x: x[-max_len:])

  train_pos = train_copy[train_copy['Label'] == 1]
  train_neg = train_copy[train_copy['Label'] == 0].copy()

  train_pos = train_pos.sample(n=len(train_neg), replace=True)

  train_pos['Sequence'] = [seq for seq in aa_to_nt(train_pos['Sequence'])]

  train_neg['Sequence'] = [seq for seq in aa_to_nt(train_neg['Sequence'])]

  train_copy = pd.concat([train_pos, train_neg])

  test_copy['Sequence'] = [seq for seq in aa_to_nt(test_copy['Sequence'])]

  train_sequences = nt_tokenizer.texts_to_sequences(train_copy['Sequence'])
  test_sequences = nt_tokenizer.texts_to_sequences(test_copy['Sequence'])

  train_sequences = tf.keras.utils.pad_sequences(train_sequences,
                                                 maxlen=max_len)
  test_sequences = tf.keras.utils.pad_sequences(test_sequences,
                                                maxlen=max_len)

  train_copy['Sequence'] = [seq for seq in train_sequences]
  test_copy['Sequence'] = [seq for seq in test_sequences]

  return train_copy, test_copy

In [14]:
l2_val = 0.001

def make_lstm_aa():
  """
  Initializes and returns a bidirectional LSTM model.

  Args:
    none
  
  Returns:
    model (keras model): the initialized model.
    
  """
  l2 = tf.keras.regularizers.L2(l2=l2_val)

  model = tf.keras.models.Sequential([
      tf.keras.layers.Embedding(22, 16, input_length=512, mask_zero=True),
      
      tf.keras.layers.Conv1D(16, 3, activation='relu', kernel_regularizer=l2),
      tf.keras.layers.MaxPool1D(2),
      
      tf.keras.layers.Conv1D(32, 3, activation='relu', kernel_regularizer=l2),
      tf.keras.layers.MaxPool1D(2),
      
      tf.keras.layers.Conv1D(64, 3, activation='relu', kernel_regularizer=l2),
      tf.keras.layers.MaxPool1D(2),
      
      tf.keras.layers.Conv1D(128, 3, activation='relu', kernel_regularizer=l2),
      tf.keras.layers.MaxPool1D(2),
      
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
      
      tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2),
      
      tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2)
  ])
  
  return model


def make_conv_aa():
  """
  Initializes and returns a 1D sequence convolutional model.

  Args:
    none
  
  Returns:
    model (keras model): the initialized model.
    
  """

  l2 = tf.keras.regularizers.L2(l2=l2_val)

  model = tf.keras.models.Sequential([
      tf.keras.layers.Embedding(22, 16, input_length=512, mask_zero=True),
      
      tf.keras.layers.Conv1D(16, 3, activation='relu', kernel_regularizer=l2),
      tf.keras.layers.MaxPool1D(2),
      
      tf.keras.layers.Conv1D(32, 3, activation='relu', kernel_regularizer=l2),
      tf.keras.layers.MaxPool1D(2),
      
      tf.keras.layers.Conv1D(64, 3, activation='relu', kernel_regularizer=l2),
      tf.keras.layers.MaxPool1D(2),
      
      tf.keras.layers.Conv1D(128, 3, activation='relu', kernel_regularizer=l2),
      tf.keras.layers.MaxPool1D(2),

      tf.keras.layers.Flatten(),
            
      tf.keras.layers.Dense(2048, activation='relu', kernel_regularizer=l2),
      
      tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2)
  ])

  return model

In [19]:
def fit_models(dataset, aug_function, model_type):
  """
  <>

  Args:
    dataset (DataFrame): the dataset of protein sequences and labels
    aug_function (function): the augmentation function used to add training data
    model_type (str): the type of model to be used
  
  Returns:
    accuracies (list): the accuracy values for each of the 5 folds
    aucs (list): the AUC values for each of the five folds
    
  """

  n_splits = 5


  # dataset_sample_frac is the fraction of the daataset which is retained when
  # it is randomly undersampled to create sampled_dataset. This is done to save
  # computational resources

  dataset_sample_frac = 0.25
  splits = KFold(n_splits=n_splits)

  accuracies = []
  aucs = []

  
  # Randomly undersamples the dataset

  sampled_dataset = dataset.sample(frac=dataset_sample_frac)

  
  # Splits the dataset into n_splits parts and fits a model on each

  for i, (train_index, test_index) in enumerate(splits.split(sampled_dataset)):
    train_set = sampled_dataset.iloc[train_index]
    test_set = sampled_dataset.iloc[test_index]

    train_aug, test_aug = aug_function(train_set, test_set)

    train_features = np.vstack(list(train_aug['Sequence']))
    test_features = np.vstack(list(test_aug['Sequence']))

    train_labels = train_aug['Label']
    test_labels = test_aug['Label']

    if model_type == 'lstm':
      model = make_lstm_aa()
    
    elif model_type == 'conv':
      model = make_conv_aa()

    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

    callbacks = [tf.keras.callbacks.EarlyStopping(patience=2, 
                                                  restore_best_weights=True
                                                  )]

    model.compile(loss='bce', 
                  optimizer=optimizer, 
                  metrics=['accuracy', tf.keras.metrics.AUC()]
                  )

    model.fit(train_features, 
              train_labels, 
              validation_data=[test_features, test_labels], 
              callbacks=callbacks, 
              epochs=50,
              verbose=0
              )

    probs = model.predict(test_features)

    accuracies.append(accuracy_score(test_labels, np.round(probs)))
    aucs.append(roc_auc_score(test_labels, probs))

  return accuracies, aucs

In [20]:
# Example of how the fit_models function is used to generate accuracy and AUC
# scores for each of 5 folds

acc, auc = fit_models(metal_ion_set, dict_aa_oversample, 'conv')

32/32 [==============================] - 0s 3ms/step


In [21]:
acc

[0.8578431372549019,
 0.873405299313052,
 0.8675171736997056,
 0.7870461236506379,
 0.8547595682041217]

In [22]:
auc

[0.7819880119880119,
 0.7281624036727332,
 0.7299328859060403,
 0.7657141917123165,
 0.7948425334535884]